In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from scipy.optimize import minimize

import json
import os

In [53]:
potential = 'AHO'
cutoff = 32
shots = None

In [54]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [55]:
min_eigenvalue

np.float64(6.182237677589772e-06)

In [56]:
cnot_pool = []
cz_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                cnot_pool.append(qml.CNOT(wires=[control, target]))
                cz_pool.append(qml.CZ(wires=[control, target]))

In [57]:
rot_pool = [qml.Rot(0.0, 0.0, 0.0, wires=x) for x in range(num_qubits)]
operator_pool = rot_pool +  cz_pool #+ cnot_pool

In [58]:
operator_pool

[Rot(0.0, 0.0, 0.0, wires=[0]),
 Rot(0.0, 0.0, 0.0, wires=[1]),
 Rot(0.0, 0.0, 0.0, wires=[2]),
 Rot(0.0, 0.0, 0.0, wires=[3]),
 Rot(0.0, 0.0, 0.0, wires=[4]),
 Rot(0.0, 0.0, 0.0, wires=[5]),
 CZ(wires=[0, 1]),
 CZ(wires=[0, 2]),
 CZ(wires=[0, 3]),
 CZ(wires=[0, 4]),
 CZ(wires=[0, 5]),
 CZ(wires=[1, 0]),
 CZ(wires=[1, 2]),
 CZ(wires=[1, 3]),
 CZ(wires=[1, 4]),
 CZ(wires=[1, 5]),
 CZ(wires=[2, 0]),
 CZ(wires=[2, 1]),
 CZ(wires=[2, 3]),
 CZ(wires=[2, 4]),
 CZ(wires=[2, 5]),
 CZ(wires=[3, 0]),
 CZ(wires=[3, 1]),
 CZ(wires=[3, 2]),
 CZ(wires=[3, 4]),
 CZ(wires=[3, 5]),
 CZ(wires=[4, 0]),
 CZ(wires=[4, 1]),
 CZ(wires=[4, 2]),
 CZ(wires=[4, 3]),
 CZ(wires=[4, 5]),
 CZ(wires=[5, 0]),
 CZ(wires=[5, 1]),
 CZ(wires=[5, 2]),
 CZ(wires=[5, 3]),
 CZ(wires=[5, 4])]

In [59]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

@qml.qnode(dev)
def circuit(params, trial_op, op_list):

    if len(op_list) > 0:
        for o, p, w, _ in op_list:
            if (o == qml.CNOT) | (o == qml.CZ):
                o(wires=w)
            else:
                o(*p, wires=w)

    op = type(trial_op)

    if (type(trial_op) == qml.CNOT) | (type(trial_op) == qml.CZ):
        op(wires=trial_op.wires)
    else:
        op(*params, wires=trial_op.wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [60]:
def cost_function(params, trial_op, op_list):
    
    params = pnp.tensor(params, requires_grad=True)
    energy = circuit(params, trial_op, op_list)

    return energy

In [67]:
num_steps = 10

x0 = np.random.uniform(0, 2 * np.pi, size=3)

op_list = []

for i in range(num_steps):

    print(f"step: {i}")

    energies = []
    e_params = []

    for trial_op in operator_pool:

        res = minimize(
                cost_function,
                x0,
                args=(trial_op, op_list),
                method= "COBYLA",
                options= {'maxiter':1000, 'tol': 1e-6}
            )
        
        energies.append(res.fun)
        e_params.append(res.x)

    min_arg = np.argmin(energies)
    min_energy = energies[min_arg]
    print(f"Min energy: {min_energy}")

    min_op = type(operator_pool[min_arg])
    min_wires = operator_pool[min_arg].wires
    min_params = e_params[min_arg]

    if (i != 0):
        if np.abs(min_energy - op_list[i-1][3]) < 1e-4:
            print("Converged")
            break

    op_list.append((min_op, min_params, min_wires, min_energy))

    

step: 0
Min energy: 0.9375000000009199
step: 1
Min energy: 0.1268240335304569
step: 2
Min energy: 0.09842000081048889
step: 3
Min energy: 0.08156474260675112
step: 4
Min energy: 0.06263045363022868
step: 5
Min energy: 0.055632970606917936
step: 6
Min energy: 0.050746622190805495
step: 7
Min energy: 0.04689439365501854
step: 8
Min energy: 0.04130958934027111
step: 9
Min energy: 0.03909646297373138


In [68]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if o == qml.CNOT:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CNOT", w.tolist()))

    elif o == qml.CZ:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CZ",w.tolist()))

    elif w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue
        else:
            last_operator[w[0]] = o
            reduced_op_list.append(("Rot",w.tolist()))
            num_params = num_params + 3
    else:
        last_operator[w[0]] = o
        reduced_op_list.append(("Rot",w.tolist()))
        num_params = num_params + 3

Operator already added previously... skipping
Operator already added previously... skipping
Operator already added previously... skipping


In [69]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "COBYLA",
        "num steps": num_steps,
        "op_list": [str(x) for x in op_list],
        "reduced_op_list": reduced_op_list,
        "num_params": num_params}

In [70]:
data

{'potential': 'AHO',
 'cutoff': 32,
 'optimizer': 'COBYLA',
 'num steps': 10,
 'op_list': ["(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.Rot'>, array([1.5570263 , 3.14159387, 4.70029003]), Wires([0]), np.float64(0.9375000000009199))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.Rot'>, array([1.90052023, 5.96410412, 6.28318569]), Wires([4]), np.float64(0.1268240335304569))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.Rot'>, array([1.16848754, 6.24919721, 3.50631795]), Wires([3]), np.float64(0.09842000081048889))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.Rot'>, array([2.60660123, 5.68218408, 5.86360212]), Wires([4]), np.float64(0.08156474260675112))",
  "(<class 'pennylane.ops.op_math.controlled_ops.CZ'>, array([1.5379991 , 3.16432192, 4.7086958 ]), Wires([4, 3]), np.float64(0.06263045363022868))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.Rot'>, array([1.35938448, 6.26499534, 4.39665491]), Wires([3]), np.

In [71]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for o, w in reduced_op_list:
        if o == "CNOT":
            qml.CNOT(wires=w)
        elif o == "CZ":
            qml.CZ(wires=w)
        else:
            num_gate_params = qml.Rot.num_params
            qml.Rot(*params[params_index:(params_index + num_gate_params)], wires=w)
            params_index = params_index + num_gate_params

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [74]:
print(qml.draw(construct_ansatz, show_matrices=False)(reduced_op_list, params))

0: ──Rot(0.81,2.03,5.43)─────────────────────────┤ ╭<𝓗(M0)>
1: ──────────────────────────────────────────────┤ ├<𝓗(M0)>
2: ──Rot(2.29,0.32,0.30)─────────────────────────┤ ├<𝓗(M0)>
3: ──Rot(1.69,2.90,4.90)─╭Z──Rot(4.55,4.43,0.81)─┤ ├<𝓗(M0)>
4: ──Rot(3.82,4.42,0.60)─╰●──Rot(0.94,6.02,2.48)─┤ ├<𝓗(M0)>
5: ──────────────────────────────────────────────┤ ╰<𝓗(M0)>


In [46]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\\"
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [52]:
reduced_op_list

[('Rot', [5]),
 ('Rot', [4]),
 ('CZ', [4, 5]),
 ('Rot', [5]),
 ('Rot', [3]),
 ('Rot', [4]),
 ('CZ', [3, 4]),
 ('Rot', [3])]

In [254]:
qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=num_qubits)

(1, 5, 3)

In [39]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if (o == qml.CNOT) | (o == qml.CZ):
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append((o,w))

    elif w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue
        else:
            last_operator[w[0]] = o
            reduced_op_list.append((o,w))
            num_params = num_params + 3
    else:
        last_operator[w[0]] = o
        reduced_op_list.append((o,w))
        num_params = num_params + 3

Operator already added previously... skipping
Operator already added previously... skipping


In [47]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for i, (o, w) in enumerate(reduced_op_list):
        if (o == qml.CNOT) | (o == qml.CZ):
            o(wires=w)
        else:
            num_gate_params = o.num_params
            o(*params[params_index:(params_index + num_gate_params)], wires=w)
            params_index = params_index + num_gate_params

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))